# Estimate non-CO2, non-CH4, non-N2O forcing

- Done as function of CO2 FFI emissions, can be done total CO2 if needed
- Include solar and volcanic for history

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import scipy.stats as st
from scipy.signal import savgol_filter
from matplotlib.lines import Line2D
from tqdm.auto import tqdm

In [ ]:
ar6_df = pd.read_csv('../data/ar6_wg3_scenarios.csv')

In [ ]:
ar6_df.Variable.unique()

In [ ]:
ar6_df

In [ ]:
ar6_df.loc[ar6_df['Variable']=='AR6 climate diagnostics|Harmonized|Emissions|CO2|Energy and Industrial Processes']

In [ ]:
y = np.empty((0))
#x = np.empty((0, 3))
x = np.empty((0, 2))

fig, ax = pl.subplots()
for year in tqdm(range(2015, 2101)):
    ffi = 0.001*ar6_df.loc[ar6_df['Variable']=='AR6 climate diagnostics|Harmonized|Emissions|CO2|Energy and Industrial Processes', str(year)]
    erf = ar6_df.loc[ar6_df['Variable']=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile', str(year)]
    time = np.ones(1202) * year
    time2 = np.ones(1202) * year * year
    y = np.append(y, erf)
    #x = np.append(x, np.column_stack([ffi, time, time2]), axis=0)
    x = np.append(x, np.column_stack([ffi, time]), axis=0)
    if year%10==0:
        sl, ic, _, _, _ = st.linregress(ffi, erf)
        shade = (2120-year)/100
        ax.scatter(ffi, erf, alpha=0.3)
        print(year, sl, ic)
        ax.plot(np.linspace(-20, 130), np.linspace(-20, 130) * sl + ic, label=year)
ax.legend()

In [ ]:
#sm_df = pd.DataFrame(x, columns=['ffi', 'year', 'year2'])
sm_df = pd.DataFrame(x, columns=['ffi', 'year'])
sm_df['erf'] = y# - 3.21
#results = ols(formula = "erf ~ ffi + year + year2", data=sm_df).fit()
results = ols(formula = "erf ~ ffi + year", data=sm_df).fit()

In [ ]:
results.summary()

In [ ]:
results.params

In [ ]:
pl.plot(
    np.arange(2015, 2101),
    (0.001 * ar6_df.loc[
        ar6_df['Variable']=='AR6 climate diagnostics|Harmonized|Emissions|CO2|Energy and Industrial Processes',
        '2015':'2100'
    ].values * results.params['ffi'] + 
    np.arange(2015, 2101)[:, None].T * results.params['year'] + 
#    np.arange(2015, 2101)[:, None].T**2 * results.params['year2'] +
    results.params['Intercept']).T
);

In [ ]:
pl.plot(
    np.arange(2015, 2101),
    ar6_df.loc[
        ar6_df['Variable']=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile',
        '2015':'2100'
    ].values.T
);

In [ ]:
ar6_erf = pd.read_csv('../data/AR6_ERF_1750-2019.csv')

In [ ]:
ar6_erf['total'] - ar6_erf['co2'] - ar6_erf['ch4'] - ar6_erf['n2o']

In [ ]:
transition = np.zeros(86)
transition[:10] = np.linspace(0.14, 0, 10)

In [ ]:
pl.plot(
    np.arange(1750, 2020),
    (ar6_erf['total'] - ar6_erf['co2'] - ar6_erf['ch4'] - ar6_erf['n2o']).values
)
pl.plot(
    np.arange(2015, 2101),
    np.median(
        ar6_df.loc[
            ar6_df['Variable']=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile',
            '2015':'2100'
        ].values.T,
        axis=1
    ) + transition
)
pl.xlim(1980, 2040)
pl.ylim(-2.5, 0.5)

In [ ]:
output = np.zeros((351))

output[:265] = (ar6_erf['total'] - ar6_erf['co2'] - ar6_erf['ch4'] - ar6_erf['n2o']).values[:265]

output[265:] = np.median(
    ar6_df.loc[
        ar6_df['Variable']=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile',
        '2015':'2100'
    ].values.T,
    axis=1
) + transition

In [ ]:
pd.DataFrame(output, index=np.arange(1750.5, 2101), columns=["residual_ERF"]).to_csv('../output/residual_ERF.csv')